In [1]:
import keras

Using TensorFlow backend.


In [3]:
keras.__version__

'2.2.4'

Using a pre-trained convnet

A common and highly effective approach to deep learning on small image datasets is to leverage a pre-trained network. A pre-trained network is simply a saved network previously trained on a large dataset, typically on a large-scale image classification task. If this original dataset is large enough and general enough, then the spatial feature hierarchy learned by the pre-trained network can effectively act as a generic model of our visual world, and hence its features can prove useful for many different computer vision problems, even though these new problems might involve completely different classes from those of the original task. For instance, one might train a network on ImageNet (where classes are mostly animals and everyday objects) and then re-purpose this trained network for something as remote as identifying furniture items in images. Such portability of learned features across different problems is a key advantage of deep learning compared to many older shallow learning approaches, and it makes deep learning very effective for small-data problems.

In our case, we will consider a large convnet trained on the ImageNet dataset (1.4 million labeled images and 1000 different classes). ImageNet contains many animal classes, including different species of cats and dogs, and we can thus expect to perform very well on our cat vs. dog classification problem.

We will use the VGG16 architecture, developed by Karen Simonyan and Andrew Zisserman in 2014, a simple and widely used convnet architecture for ImageNet. Although it is a bit of an older model, far from the current state of the art and somewhat heavier than many other recent models, we chose it because its architecture is similar to what you are already familiar with, and easy to understand without introducing any new concepts. This may be your first encounter with one of these cutesie model names -- VGG, ResNet, Inception, Inception-ResNet, Xception... you will get used to them, as they will come up frequently if you keep doing deep learning for computer vision.

There are two ways to leverage a pre-trained network: feature extraction and fine-tuning. We will cover both of them. Let's start with feature extraction.

# Feature Extraction

Feature extraction consists of using the representations learned by a previous network to extract interesting features from new samples. These features are then run through a new classifier, which is trained from scratch.

In [5]:
from keras.applications import VGG16
conv_base=VGG16(weights='imagenet',
             include_top=False,
             input_shape=(150,150,3)
             )
 

58892288/58889256 [==============================] - 79s 1us/step


In [7]:
from keras.applications import ResNet50
model = ResNet50(weights='imagenet')

In [8]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Extending the model we have (conv_base) by adding Dense layers on top, and running the whole thing end-to-end on the input data. This allows us to use data augmentation, because every input image is going through the convolutional base every time it is seen by the model. However, for this same reason, this technique is far more expensive than the first one.

In [11]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir='E:/deeplearningtry/dogs-vs-cats/data'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


In [17]:
datagen=ImageDataGenerator(rescale=1./255)
batch_size=20
def extract_features(dic,count):
    features=np.zeros(shape=(count,4,4,512))
    labels=np.zeros(shape=(count))
    gen=datagen.flow_from_directory(
    dic,
    target_size=(150,150),
    batch_size = batch_size,
    class_mode = 'binary')
    i=0
    for inputs,features_labels in gen:
        features_batch=conv_base.predict(inputs)
        features[i*batch_size:(i+1)*batch_size]=features_batch
        labels[i*batch_size:(i+1)*batch_size]=features_labels
        i+=1
        if i*batch_size>=count:
            break
    return features,labels

train_features,train_labels=extract_features( train_dir,2000)

validation_features,validation_labels=extract_features( validation_dir,1000)

test_features,test_labels=extract_features( test_dir,1000)
    

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


要想收敛到同一个最优点，使用整个样本集时，虽然迭代次数少，但是每次迭代的时间长，耗费的总时间是大于使用少量样本多次迭代的情况的。
实际上，工程上在使用GPU训练时，跑一个样本花的时间与跑几十个样本甚至几百个样本的时间是一样的！当然得益于GPU里面超多的核，超强的并行计算能力啦。因此，在工程实际中，从收敛速度的角度来说，小批量的样本集是最优的，也就是我们所说的mini-batch。这时的batch size往往从几十到几百不等，但一般不会超过几千。

样本量少的时候会带来很大的方差，而这个大方差恰好会导致我们在梯度下降到很差的局部最优点（只是微微凸下去的最优点）和鞍点的时候不稳定，一不小心就因为一个大噪声的到来导致炸出了局部最优点。
与之相反的，当样本量很多时，方差很小，对梯度的估计要准确和稳定的多，因此反而在差劲的局部最优点和鞍点时反而容易自信的呆着不走了，从而导致神经网络收敛到很差的点上，跟出了bug一样的差劲。

batch的size设置的不能太大也不能太小，因此实际工程中最常用的就是mini-batch，一般size设置为几十或者几百。
对于二阶优化算法，减小batch换来的收敛速度提升远不如引入大量噪声导致的性能下降，因此在使用二阶优化算法时，往往要采用大batch哦。此时往往batch设置成几千甚至一两万才能发挥出最佳性能。
GPU对2的幂次的batch可以发挥更佳的性能，因此设置成16、32、64、128...时往往要比设置为整10、整100的倍数时表现更优


In [18]:
train_features=np.reshape(train_features,(2000,4*4*512))

In [20]:
validation_features=np.reshape(validation_features,(1000,4*4*512))
test_features=np.reshape(test_features,(1000,4*4*512))

In [19]:
from keras import models,layers,optimizers

In [21]:
from keras.layers import Dense,Dropout


In [24]:
model=models.Sequential()
model.add(Dense(256,activation='relu',input_dim=4*4*512))
#model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])
history=model.fit(train_features,train_labels,epochs=50,batch_size=32,
                 validation_data=(validation_features,validation_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/50
2000/2000 [==============================] - 2s 911us/step - loss: 0.5326 - acc: 0.7585 - val_loss: 0.4346 - val_acc: 0.8420
Epoch 2/50
2000/2000 [==============================] - 1s 620us/step - loss: 0.3765 - acc: 0.8570 - val_loss: 0.3660 - val_acc: 0.8530
Epoch 3/50
2000/2000 [==============================] - 1s 618us/step - loss: 0.3108 - acc: 0.8895 - val_loss: 0.3211 - val_acc: 0.8720
Epoch 4/50
2000/2000 [==============================] - 1s 618us/step - loss: 0.2726 - acc: 0.8975 - val_loss: 0.3047 - val_acc: 0.8740
Epoch 5/50
2000/2000 [==============================] - 1s 622us/step - loss: 0.2448 - acc: 0.9135 - val_loss: 0.2973 - val_acc: 0.8710
Epoch 6/50
2000/2000 [==============================] - 1s 623us/step - loss: 0.2221 - acc: 0.9260 - val_loss: 0.2940 - val_acc: 0.8690
Epoch 7/50
2000/2000 [==============================] - 1s 626us/step - loss: 0.2065 - acc: 0.9260 - val_loss: 0.2758 - val_acc: 0.8820


We reach a validation accuracy of about 90%, much better than what we could achieve in the previous section with our small model trained from scratch. However, our plots also indicate that we are overfitting almost from the start -- despite using dropout with a fairly large rate. This is because this technique does not leverage data augmentation, which is essential to preventing overfitting with small image datasets.

Now, let's review the second technique we mentioned for doing feature extraction, which is much slower and more expensive, but which allows us to leverage data augmentation during training: extending the conv_base model and running it end-to-end on the inputs. Note that this technique is in fact so expensive that you should only attempt it if you have access to a GPU: it is absolutely intractable on CPU. If you cannot run your code on GPU, then the previous technique is the way to go.

Because models behave just like layers, you can add a model (like our conv_base) to a Sequential model just like you would add a layer. So you can do the following:

In [25]:
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 4


In [26]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [28]:
conv_base.trainable = False

In [29]:
print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 4


In [30]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
 - 190s - loss: 0.5860 - acc: 0.6965 - val_loss: 0.4602 - val_acc: 0.8010
Epoch 2/30
 - 193s - loss: 0.4759 - acc: 0.7735 - val_loss: 0.3754 - val_acc: 0.8440
Epoch 3/30
 - 201s - loss: 0.4234 - acc: 0.8205 - val_loss: 0.3438 - val_acc: 0.8490
Epoch 4/30
 - 198s - loss: 0.4046 - acc: 0.8200 - val_loss: 0.3239 - val_acc: 0.8550
Epoch 5/30
 - 197s - loss: 0.3767 - acc: 0.8315 - val_loss: 0.3133 - val_acc: 0.8610
Epoch 6/30
 - 198s - loss: 0.3672 - acc: 0.8435 - val_loss: 0.3034 - val_acc: 0.8560
Epoch 7/30
 - 196s - loss: 0.3596 - acc: 0.8440 - val_loss: 0.3147 - val_acc: 0.8680
Epoch 8/30
 - 195s - loss: 0.3442 - acc: 0.8440 - val_loss: 0.2973 - val_acc: 0.8750
Epoch 9/30
 - 195s - loss: 0.3339 - acc: 0.8525 - val_loss: 0.2835 - val_acc: 0.8730
Epoch 10/30
 - 197s - loss: 0.3371 - acc: 0.8485 - val_loss: 0.2818 - val_acc: 0.8750
Epoch 11/30
 - 196s - loss: 0.3280 - acc: 0.8620 - val_loss: 0.28

We have stated before that it was necessary to freeze the convolution base of VGG16 in order to be able to train a randomly initialized classifier on top. For the same reason, it is only possible to fine-tune the top layers of the convolutional base once the classifier on top has already been trained. If the classified wasn't already trained, then the error signal propagating through the network during training would be too large, and the representations previously learned by the layers being fine-tuned would be destroyed. Thus the steps for fine-tuning a network are as follow:

1) Add your custom network on top of an already trained base network.
2) Freeze the base network.
3) Train the part you added.
4) Unfreeze some layers in the base network.
5) Jointly train both these layers and the part you added.
We have already completed the first 3 steps when doing feature extraction. Let's proceed with the 4th step: we will unfreeze our conv_base, and then freeze individual layers inside of it.

As a reminder, this is what our convolutional base looks like:

In [36]:
conv_base.trainable=True
set_trainable=False
for layer in conv_base.layers:
    if layer.name=='block5_conv1':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False
        
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)
        

Epoch 1/100
 10/100 [==>...........................] - ETA: 2:27 - loss: 0.2924 - acc: 0.8550

KeyboardInterrupt: 

import numpy
import tensorflow as tf
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))




# 测试是GPU在跑还是CPU

In [37]:
mport numpy import tensorflow as tf a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a') b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b') c = tf.matmul(a, b) sess = tf.Session(config=tf.ConfigProto(log_device_placement=True)) print(sess.run(c))

SyntaxError: invalid syntax (<ipython-input-37-adcf59796f82>, line 1)

In [38]:
import numpy
import tensorflow as tf
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

[[22. 28.]
 [49. 64.]]
